# Examen – Modelos de Ensamble: Árboles y Boosting

**Instrucciones**: Justifica cada respuesta de manera clara. Usa fórmulas donde aplique y sé específico en tus argumentos.

---

## Sección I: Árboles de decisión (15 puntos)

### 1. (5 pts)  
**Explica cómo se construye un árbol y qué criterio usa para decidir los splits. Explica tanto para el caso de clasificación como de regresión.**

Un modelo el cual utiliza varios niveles, donde en cada nivel existe una caractrística que se evalua a través de un umbral, va dividiendo los datos en los que son mayor o menor al umbral definido en la característica.


### Regresión

Este algoritmo usa todas las variables y los splits posibles (umbrales) y elige el umbral en cada variable que más reduzca la varianza.


$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

Varianza de cada rama ponderada por el número de gente de cada una de las ramas.

### Clasificación

De igual forma que en la regresión, el modelo usa todas las variables y prueba todos los umbrales posibles, eligiendo el que cause una mayor reducción de impureza, la cual se calcula a través del Gini o de la Entriopía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$


$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

Impureza de cada rama ponderada por el número de personas en cada una de ellas.

### 2. (5 pts)  
**Da un ejemplo de sobreajuste en un árbol de decisión. Explica cómo se podría evitar sin necesidad de usar ensambles.**

Un caso común es cuando el max_depth es muy grande, ya que el árbol se va expandiendo cada vez más haciendo muchos niveles y muchos splits lo cual provoca que termine memorizando los datos. La forma más sencilla de evitar esto es con un cross validation que pruebe distintos depths de árbol para encontrar el mejor, además de mover otros hiperparámetros.

### 3. (5 pts)  
**Si te fijas, en clase nunca hicimos escalamiento (`StandardScaler`). ¿Por qué los árboles no lo necesitan?**

El `StandardScaler` sirve para poner en una misma escala tus variables, esto es útil para modelos de KNN que usan distancias, y puede ser útil en regresiones para dar como la misma importancia a las variables, sin embargo, por la construcción de los árboles esto no es necesario ya que comparas sobre el valor real de tus datos para definir tus reglas de decisión (umbrales) que son los que más reducen la varianza. Al hacer splits sobre tus valores no es de ayuda escalarlas ya que con los valores originales se pueden establecer los splits adecuados.

---

## Sección II: Random Forest (20 puntos)

### 4. (10 pts)  
**Explica cómo funciona un Random Forest. ¿En qué se basa? ¿Por qué es una buena idea?**

Utilizas la técnica de bootsrap para hacer muestreos aleatorios de tus datos, con cada uno de los muestreos entrenas un árbol de decisión distinto logrando así un buen entrenamiento con la construcción de múltiples árboles de decisión. Cada uno de estos modelos elige distintas variables de forma aleatoria, realiza sus predicciones, y el promedio de estas sería la predicción utilizada por el algoritmo de Random Forest o la más usada en caso de clasificación. Esto es útil ya que de esta forma puedes probar muchos datos distintos por el muestreo aleatorio para entrenar de mejor manera, además de utilizar distintas variables y splits, esto ayuda a dar mejores resultados que un solo árbol pues hace un entrenamiento más robusto.

### 5. (10 pts)  
**Menciona dos ventajas y dos desventajas del Random Forest comparado con un solo árbol. Sé específico, no generalices.**

`Ventajas`:
- Menos probabilidad de hacer overfitting por el uso de bootsrap ya que añada aleatoriedad y usa todos los resultados.
- Mejores predicciones ya que combina los resultados me múltiples modelos entrenados con una gran variedad de datos.

`Desventajas`:
- Computacionalmente costoso, es decir, tarda en correr por entrenar muchos árboles.
- Su interpretación no es ta rápida y directa como la de un árbol de decisión.

---

## Sección III: Gradient Boosting (25 puntos)

### 6. (10 pts)  
**Explica, paso a paso, cómo funciona el algoritmo de **`Gradient Boosting`**. Incluye el concepto de residuales y cómo se minimiza la pérdida en cada iteración.**

Inicias tu modelo con un valor constante (puede ser el promedio de tus datos por ejemplo).

Después calculas los residuales de este modelo con un loop a través de los árboles (valor real - valor predicho), de manera formal:

$$
r_{im} = - \left[ \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right]_{F(x)=F_{m-1}(x)} \quad \text{para } i = 1 \dots n
$$

Una vez hecho esto se ajusta un árbol de regresión para predecir los residuales y para cada una de las regiones obtienes:

$$
\gamma_{jm} = \arg\min_\gamma \sum_{x_i \in R_{jm}} L(y_i, F_{m-1}(x_i) + \gamma)
$$

Por último se actualiza el modelo que es el modelo anterior más el nuevo árbol ajustado por el learning rate:

$$F_m(x) = F_{m - 1}(x) + \nu (\text{nuevo árbol})$$

El proceso se repite hasta converger, haciendo el error cada vez más chico. Similar al descenso en gradiente.



### 7. (15 pts)  
**¿Cuál es la diferencia entre Gradient Boosting y Random Forest en términos de cómo combinan los árboles?** 

Random Forest usa muetreos aleatorios para entrenar muchos árboles distintos y usa las predicciones de cada uno para la predicción final, ya sea el promedio y los más usada. Gradient Boosting obtiene los residuales (real - predicho) y ajusta un árbol de regresión para poder predecir los residuales, el promedio de estos residuales se usan como predicción para el siguiente árbol y actualiza el modelo usando el anteriror más el nuevo árbol multiplicado por un ajuste.

Por lo tanto funcionan muy distinto ya que uno hace muchos árboles y promedia las predicciones (Random Forest) y otro es más como descenso en gradiente donde usa los residuales y su predición de estos para actualizar el modelo.

---


## Sección IV: XGBoost (20 puntos)

### 8. (10 pts)  
**Explica cómo XGBoost optimiza el proceso de boosting usando una expansión de Taylor de segundo orden.**

### 9. (5 pts)  
**¿Qué es el *similarity score*? ¿Qué es el *output value*? ¿De dónde salen estas fórmulas y cuál es su interpretación?**

### 10. (5 pts)  
**XGBoost y otros modelos de gradient boosting permiten evaluar la importancia de las variables con diferentes métricas: `weight` y `gain`. Explica qué representa cada una. ¿Cuál crees que es más útil para interpretar un modelo y por qué?**

---

## Sección V: LightGBM y CatBoost (15 puntos)

### 10. (5 pts)  
**Explica qué es **histogram-based splitting** y cómo lo implementa LightGBM para ganar velocidad.**

### 11. (5 pts)  
**¿Qué problema específico resuelve CatBoost respecto al manejo de variables categóricas? ¿Cómo lo hace?**

### 12. (5 pts)  
**Compara LightGBM y CatBoost: ¿cuándo usarías uno sobre el otro? Sé claro y justifica en base a tipo de datos, velocidad o precisión.**

## Sección VI: Power Analysis (5 puntos)

### 13. (5 pts)  
**¿Qué es un *power analysis* y para qué sirve? ¿En qué contexto lo hemos usado en clase y por qué es importante antes de correr un experimento?**

